In [1]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch import nn

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
custom_transform = transforms.Compose([
    transforms.Resize(size=(224,224)), # bcs resnet model is trained on 224*224 images
    transforms.ToTensor(),
    transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225)) # inc the img size so use this
])

In [4]:
train_data = datasets.CIFAR10(root="/content/datasets", train=True, download=True, transform=custom_transform)
test_data = datasets.CIFAR10(root="/content/datasets", train=False, download=True, transform=custom_transform)

100%|██████████| 170M/170M [00:04<00:00, 41.9MB/s]


Extracting /content/datasets/cifar-10-python.tar.gz to /content/datasets
Files already downloaded and verified


In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

# hugging face surve opensource NN model
* use pre trained resnet model

In [6]:
# download resnet model (18 convolutional layers)
resnet18 = models.resnet18(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 174MB/s]


In [7]:
print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
# before we do transfer learning freeze the feature layers
# controls whether or not gradients are calculated for those parameters during the training process
for params in resnet18.parameters():
    params.required_grad = False

In [9]:
# update the resnet18 model
resnet18.fc = nn.Linear(resnet18.fc.in_features, 10) # 1000 classes ->> 10 classes

In [10]:
print(resnet18.fc)

Linear(in_features=512, out_features=10, bias=True)


# create Model

In [11]:
resnet18 = resnet18.to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet18.fc.parameters(), lr=0.001)

In [13]:
resnet18.train() # originally resnet18 was train in 3.5 days

for epoc in range(50):
    total_loss = 0

    for train_images, train_labels in train_loader:
        train_images = train_images.to(device)
        train_labels = train_labels.to(device)

        optimizer.zero_grad()
        output = resnet18(train_images)
        loss = criterion(output, train_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"epoc: {epoc+1}/20, total loss is: {total_loss}")

epoc: 1/20, total loss is: 2402.987946510315
epoc: 2/20, total loss is: 1556.2049349546432
epoc: 3/20, total loss is: 1344.4314031600952
epoc: 4/20, total loss is: 1242.9890680909157
epoc: 5/20, total loss is: 1177.8116545677185


KeyboardInterrupt: 

# model evalution

In [ ]:
correct = 0
total = 0
resnet18.eval()

with torch.no_grad():

    for test_images, test_labels in test_loader:
        test_images = test_images.to(device)
        test_labels = test_labels.to(device)

        output = resnet18(test_images)
        pred_labels = torch.argmax(output, dim=1)
        correct += (pred_labels == test_labels).sum().item()
        total += test_labels.size(0) # batch_size = 32

    print(f"Model Accuracy is: {correct/total*100:0.2f}%")